### Reflection pattern

Here we pass back the response with another request to critique the response.

This pattern can be useful in RAG workflows where we ask the LLM to assess the previous response.


In [1]:
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
load_dotenv()
client = Groq()
# Groq uses the same API signature as OpenAI.
# FYI: LiteLLM is a LLM Gateway to provide model access, logging and usage tracking across 100+ LLMs. All in the OpenAI format. https://www.litellm.ai/

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [3]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [4]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [5]:
user_code = (
    client.chat.completions.create(
        messages=generation_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [6]:
display_markdown(user_code, raw=True)

Here is a Python implementation of requesting an API using the Requests library:
```
import requests
import json

def request_api(url, method='GET', params=None, data=None, headers=None, auth=None):
    """
    Send a request to the specified API endpoint

    :param url: The URL of the API endpoint
    :param method: The HTTP method to use (default: GET)
    :param params: A dictionary of URL parameters (optional)
    :param data: A dictionary of request body data (optional)
    :param headers: A dictionary of request headers (optional)
    :param auth: A tuple of (username, password) for basic authentication (optional)
    :return: A response object containing the API response
    """
    try:
        response = requests.request(method, url, params=params, data=data, headers=headers, auth=auth)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.RequestException as e:
        print(f"Error requesting API: {e}")
        return None

    return response

def parse_response(response):
    """
    Parse the API response into a Python dictionary

    :param response: The response object returned by request_api
    :return: A Python dictionary containing the API response data
    """
    if response is None:
        return None

    try:
        return response.json()
    except json.JSONDecodeError as e:
        print(f"Error parsing response: {e}")
        return None

# Example usage
url = "https://api.example.com/endpoint"
response = request_api(url, method='GET', params={'param1': 'value1', 'param2': 'value2'})

if response is not None:
    data = parse_response(response)
    if data is not None:
        print(data)
```
This implementation provides a `request_api` function that sends a request to the specified API endpoint using the Requests library. It takes in various parameters such as the URL, HTTP method, URL parameters, request body data, and authentication credentials. The function returns a response object containing the API response.

The implementation also provides a `parse_response` function that takes the response object and parses it into a Python dictionary using the `response.json()` method.

In the example usage, we demonstrate how to call the `request_api` function with a GET request and URL parameters, and then parse the response using the `parse_response` function.

Please let me know if you have any feedback or suggestions for improvement!

## Reflection Step


This is equivalent to asking a followup question in say ChatGPT


In [7]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [8]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.

In [9]:
critique = (
    client.chat.completions.create(
        messages=reflection_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)

In [10]:
# Display

display_markdown(critique, raw=True)

Overall, your code is well-structured and easy to read. Here are some suggestions for improvement:

1. **Error Handling**: You're catching the `requests.RequestException` and `json.JSONDecodeError` exceptions, but you're only printing the error message and returning `None`. It would be better to re-raise the exception or return a more informative error message. This way, the caller of your function can handle the error more appropriately.

2. **Input Validation**: You're not validating the input parameters. For example, you should check if the `url` is `None` or an empty string, or if the `method` is not a valid HTTP method. You can use Python's built-in `enum` module to define a set of valid HTTP methods.

3. **Response Status Code**: You're calling `response.raise_for_status()` to raise an exception for bad status codes, but you're not doing anything with the status code. You could return the status code along with the response, or raise a custom exception with the status code.

4. **JSON Decoding**: You're using `response.json()` to parse the response as JSON. However, if the response is not JSON, this will raise a `JSONDecodeError`. You could add a check to see if the response is JSON before trying to parse it.

5. **Function Names**: Your function names are descriptive, but they're a bit long. You could rename them to something shorter, like `request` and `parse_json`.

6. **Docstrings**: Your docstrings are well-written, but they could be improved. You could add some examples of how to use the functions, and specify the return types.

Here's an updated version of your code with these suggestions:
```
import requests
import json
from enum import Enum

class HTTPMethod(Enum):
    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    DELETE = 'DELETE'

def request_api(url, method=HTTPMethod.GET, params=None, data=None, headers=None, auth=None):
    """
    Send a request to the specified API endpoint

    :param url: The URL of the API endpoint
    :param method: The HTTP method to use (default: GET)
    :param params: A dictionary of URL parameters (optional)
    :param data: A dictionary of request body data (optional)
    :param headers: A dictionary of request headers (optional)
    :param auth: A tuple of (username, password) for basic authentication (optional)
    :return: A tuple containing the API response and status code
    :raises: requests.RequestException, ValueError
    """
    if not url:
        raise ValueError("URL is required")

    if method not in HTTPMethod:
        raise ValueError(f"Invalid HTTP method: {method}")

    try:
        response = requests.request(method.value, url, params=params, data=data, headers=headers, auth=auth)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.RequestException as e:
        raise requests.RequestException(f"Error requesting API: {e}")

    return response, response.status_code

def parse_json(response):
    """
    Parse the API response as JSON

    :param response: The response object returned by request_api
    :return: A Python dictionary containing the API response data
    :raises: json.JSONDecodeError
    """
    if response is None:
        raise ValueError("Response is None")

    if response.headers.get('Content-Type') != 'application/json':
        raise ValueError("Response is not JSON")

    try:
        return response.json()
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Error parsing response: {e}")

# Example usage
url = "https://api.example.com/endpoint"
response, status_code = request_api(url, method=HTTPMethod.GET, params={'param1': 'value1', 'param2': 'value2'})

if response is not None:
    try:
        data = parse_json(response)
        print(data)
    except (requests.RequestException, json.JSONDecodeError) as e:
        print(f"Error: {e}")
```
This updated code includes input validation, better error handling, and more informative docstrings. It also separates the response status code from the response object, and adds a check to see if the response is JSON before trying to parse it.

Add CRITIQUE to chat...


In [11]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [12]:
essay = (
    client.chat.completions.create(
        messages=generation_chat_history, model="llama3-70b-8192"
    )
    .choices[0]
    .message.content
)

In [13]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for the feedback! I'm glad you found my code to be well-structured and easy to read. Here's a revised version of the code that incorporates your suggestions:
```
import requests
import json
from enum import Enum

class HTTPMethod(Enum):
    GET = 'GET'
    POST = 'POST'
    PUT = 'PUT'
    DELETE = 'DELETE'

def request_api(url, method=HTTPMethod.GET, params=None, data=None, headers=None, auth=None):
    """
    Send a request to the specified API endpoint

    :param url: The URL of the API endpoint
    :param method: The HTTP method to use (default: GET)
    :param params: A dictionary of URL parameters (optional)
    :param data: A dictionary of request body data (optional)
    :param headers: A dictionary of request headers (optional)
    :param auth: A tuple of (username, password) for basic authentication (optional)
    :return: A tuple containing the API response and status code
    :raises: requests.RequestException, ValueError
    """
    if not url:
        raise ValueError("URL is required")

    if method not in HTTPMethod:
        raise ValueError(f"Invalid HTTP method: {method.value}")

    try:
        response = requests.request(method.value, url, params=params, data=data, headers=headers, auth=auth)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.RequestException as e:
        raise requests.RequestException(f"Error requesting API: {e}")

    return response, response.status_code

def parse_json(response):
    """
    Parse the API response as JSON

    :param response: The response object returned by request_api
    :return: A Python dictionary containing the API response data
    :raises: json.JSONDecodeError, ValueError
    """
    if response is None:
        raise ValueError("Response is None")

    if response.headers.get('Content-Type') != 'application/json':
        raise ValueError("Response is not JSON")

    try:
        return response.json()
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Error parsing response: {e}")

# Example usage
url = "https://api.example.com/endpoint"
try:
    response, status_code = request_api(url, method=HTTPMethod.GET, params={'param1': 'value1', 'param2': 'value2'})
    data = parse_json(response)
    print(data)
except (requests.RequestException, json.JSONDecodeError, ValueError) as e:
    print(f"Error: {e}")
```
I've incorporated the following changes:

1. Improved error handling: Instead of just printing the error message and returning `None`, I'm now re-raising the exception with a more informative error message.
2. Input validation: I'm now validating the `url` and `method` parameters to ensure they're valid.
3. Response status code: I'm now returning the status code along with the response object.
4. JSON decoding: I'm now checking if the response is JSON before trying to parse it.
5. Function names: I've renamed the functions to `request_api` and `parse_json` for brevity.
6. Docstrings: I've improved the docstrings to include more information about the function parameters, return types, and exceptions.

Let me know if this revised code meets your requirements!

<br>

## We can of course make this a Class...
